In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone, date
from scipy.signal import find_peaks
import MetaTrader5 as mt5
from zoneinfo import ZoneInfo
import pytz
import time
from IPython.display import clear_output
from pandas import Timestamp
import sell_price_action_detector
import buy_price_action_detector
import order_manager

In [3]:
# Initialize MetaTrader 5 connection
order_manager.log_in_to_mt5()

Connected to MT5 successfully


In [4]:
#DROP THE UNNCESSARY COLUMNS IN THE DATAFRAME AND CONVERTING THE TIME TO HUMAN READABLE FORMAT
def df_convert_hr(rates_frame_input):
    rates_frame_input['time'] = pd.to_datetime(rates_frame_input['time'], unit='s')
    rates_frame_input.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    rates_frame_input["time"] = pd.to_datetime(rates_frame_input["time"])
    return rates_frame

def append_new_data(rates_frame, rates_frame_to_append):
    rates_frame = pd.concat([rates_frame, rates_frame_to_append]).drop_duplicates().reset_index(drop=True)
    return rates_frame

def download_recent_OHLCdata():
    global symbol
    timeframe = mt5.TIMEFRAME_M1
    
    #GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
    utc_from = datetime.now() - timedelta(minutes=5) + timedelta(hours=2)
    utc_to = datetime.now() + timedelta(hours=2) - timedelta(minutes=1)
    startdate=utc_from.strftime('%Y%m%d')
    enddate=utc_to.strftime('%Y%m%d')
    
    # Get the OHLC data FOR TODAY AND YESTERDAY
    rates_5m = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
    # Check if the data is downloaded successfully
    if rates is None:
        print("No data available, error code =", mt5.last_error())
    else:
        # Convert the data to a pandas DataFrame
        rates_frame_to_append = pd.DataFrame(rates_5m)
        df_convert_hr(rates_frame_to_append)
        return rates_frame_to_append

def download_and_display_updated_values():
    global rates_frame, N
    rates_frame_to_append = download_recent_OHLCdata()
    rates_frame = append_new_data(rates_frame, rates_frame_to_append)
    print(f"Number of trades executed for today: {N}")
    print("  ")
    print(f"{symbol} OHLC prices from the last 30 minutes :")
    print("____________________________________________________________________________")
    print(rates_frame.tail(30))
    return

In [5]:
#Detect the entry drills
def check_buy_entry_drill():
    if buy_tp == "-":
        Buy = False
        SL_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = buy_price_action_detector.create_empty_time_price_df()
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
        
    global rates_frame, Last_110, rates_frame_p3, p3_rates_frame

    #check if pattern occured during pre-london session
    pre_ldn_session = 9 <= LowestPoint["time"].to_pydatetime().hour < 10
    #check if pattern occured during london session:
    ldn_session = 10 <= LowestPoint["time"].to_pydatetime().hour < 14
    #check if pattern occured during pre-ny session
    pre_ny_session = 14 <= LowestPoint["time"].to_pydatetime().hour < 15
    #check if pattern occured during ny session
    ny_session = 15 <= LowestPoint["time"].to_pydatetime().hour < 19
    
    condition1 = ldn_session and (ldn_low == LowestPoint["low"])
    condition2 = ny_session and (ny_low == LowestPoint["low"])
    condition3 = pre_ldn_session and (pre_ldn_low == LowestPoint["low"])
    condition4 = pre_ny_session and (pre_ny_low == LowestPoint["low"])
    is_OTE = condition1 or condition2 or condition3 or condition4
    
    if is_OTE:
        print('Price is at OTE of the session!')
        (
            Buy, SL_price, 
            TP1_price, OB_size,
            Entry_price, SL_size,
            p1, p2, p2_bos, p3, p4
        ) = buy_price_action_detector.validate_and_buy(rates_frame_p3, p3_rates_frame, buy_tp)
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
    else:
        print("It is either price is not at the OTE of the session or it is not time to trade!")
        Buy = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = buy_price_action_detector.create_empty_time_price_df()
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4

def check_sell_entry_drill():
    if sell_tp == "-":
        Sell = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = sell_price_action_detector.create_empty_time_price_df()
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
        
    global rates_frame, Last_110, rates_frame_p3, p3_rates_frame
    
    #check if pattern occured during pre-london session
    pre_ldn_session = 9 <= HighestPoint["time"].to_pydatetime().hour < 10
    #check if pattern occured during london session:
    ldn_session = 10 <= HighestPoint["time"].to_pydatetime().hour < 14
    #check if pattern occured during pre-ny session
    pre_ny_session = 14 <= HighestPoint["time"].to_pydatetime().hour < 15
    #check if pattern occured during ny session
    ny_session = 15 <= HighestPoint["time"].to_pydatetime().hour < 19

    condition1 = ldn_session and (ldn_high == HighestPoint["high"])
    condition2 = ny_session and (ny_high == HighestPoint["high"])
    condition3 = pre_ldn_session and (pre_ldn_high == HighestPoint["high"])
    condition4 = pre_ny_session and (pre_ny_high == HighestPoint["high"])
    is_OTE = condition1 or condition2 or condition3 or condition4
    
    if is_OTE:
        print('Price is at OTE of the session!')
        (
            Sell, SL_price, TP1_price, 
            OB_size, Entry_price, SL_size, 
            p1, p2, p2_bos, p3, p4 
        ) = sell_price_action_detector.validate_and_sell(rates_frame_p3, p3_rates_frame, sell_tp)
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
    else:
        print("It is either price is not at the OTE of the session or it is not time to trade!")
        Sell = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = sell_price_action_detector.create_empty_time_price_df()
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4

In [6]:
symbol = "GBPUSD"
timeframe = mt5.TIMEFRAME_M1

#GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
utc_from = datetime.now() - timedelta(days=1) + timedelta(hours=2)
utc_to = datetime.now() - timedelta(minutes=1) + timedelta(hours=2)
startdate=utc_from.strftime('%Y%m%d')
enddate=utc_to.strftime('%Y%m%d')

# Get the OHLC data FOR TODAY AND YESTERDAY
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
# Check if the data is downloaded successfully
if rates is None:
    print("No data available, error code =", mt5.last_error())
else:
    # Convert the data to a pandas DataFrame
    rates_frame = pd.DataFrame(rates)
    rates_frame = df_convert_hr(rates_frame)
    print(rates_frame)

                    time     open     high      low    close
0    2025-01-22 07:54:00  1.23475  1.23475  1.23455  1.23455
1    2025-01-22 07:55:00  1.23456  1.23465  1.23434  1.23434
2    2025-01-22 07:56:00  1.23435  1.23442  1.23425  1.23426
3    2025-01-22 07:57:00  1.23425  1.23433  1.23423  1.23433
4    2025-01-22 07:58:00  1.23434  1.23439  1.23423  1.23425
...                  ...      ...      ...      ...      ...
1434 2025-01-23 07:48:00  1.23123  1.23123  1.23113  1.23114
1435 2025-01-23 07:49:00  1.23114  1.23124  1.23114  1.23119
1436 2025-01-23 07:50:00  1.23119  1.23123  1.23119  1.23123
1437 2025-01-23 07:51:00  1.23120  1.23123  1.23120  1.23122
1438 2025-01-23 07:52:00  1.23123  1.23133  1.23123  1.23133

[1439 rows x 5 columns]


In [7]:
try:
    current_price = float(rates_frame["close"].tail(1).iloc[0])
    print(f"The current price of {symbol} is about: {current_price}.")

    while True:
        sell_tp = input("Enter the sell TP: ").strip()
        try: 
            # Check for a dash
            if sell_tp == "-":
                print("You are only looking to buy.")
                BEARISH_BIAS = False
                break
        
            sell_tp = float(sell_tp)
            if sell_tp >= current_price:
                print(f"Enter a price that is less than the current price of {symbol}.")
                continue
            value = float(sell_tp)
            print(f"You entered a TP level: {value}")
            BEARISH_BIAS = True
            break
        except ValueError:
            print(f"Invalid input. Please enter a dash '-' or a valid price of {symbol}.")
            continue
    
    while True:
        buy_tp = input("Enter the buy TP: ").strip()
        try:
            if buy_tp == "-":
                print("You are only looking to sell.")
                BULLISH_BIAS = False
                break
            
            buy_tp = float(buy_tp) 
            if buy_tp <= current_price:
                print(f"Enter a price that is more than the current price of {symbol}.")
                continue
                
            value = float(buy_tp)
            print(f"You entered a TP level: {value}")
            BULLISH_BIAS = True
            break
            
        except ValueError:
            print(f"Invalid input. Please enter a dash '-' or a valid price of {symbol}.")
            continue
            
except IndexError as err1:
    print(f'IndexError : {err1} ')
except NameError as err2:
    print(f'Name error 1 : {err2} ')

The current price of GBPUSD is about: 1.23133.


Enter the sell TP:  1.20996


You entered a TP level: 1.20996


Enter the buy TP:  -


You are only looking to sell.


In [ ]:
#number of trades for today
N = 0
date_today = date.today()

while N < 3:
    if N >= 3:
        break
    else:
        clear_output(wait=True)
        #HAVE THE DATAFRAME FOR THE CURRENT LONDON SESSION and LOOK FOR THE LONDON HIGH AND LOW:
        pre_ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 9) & (rates_frame['time'].dt.hour < 10)]
        if len(pre_ldn_rates_frame) != 0:    
            pre_ldn_high = round(pre_ldn_rates_frame["high"].max(),5)
            pre_ldn_low = round(pre_ldn_rates_frame["low"].min(),5)
            print("__________________________________")
            print(f'Pre-London high: {pre_ldn_high}')
            print(f'Pre-London low:  {pre_ldn_low}')
        else: 
            pass

        ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 10) & (rates_frame['time'].dt.hour < 14)]
        if len(ldn_rates_frame) != 0:
            ldn_high = round(ldn_rates_frame["high"].max(),5)
            ldn_low = round(ldn_rates_frame["low"].min(),5)
            print(f'\nLondon high:.....{ldn_high}')
            print(f'London low:......{ldn_low}')
        else:
            pass

        #HAVE THE DATAFRAME FOR THE CURRENT NEW YORK SESSION and LOOK FOR THE NY HIGH AND LOW: 
        pre_ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 14) & (rates_frame['time'].dt.hour < 15)]
        if len(pre_ny_rates_frame) != 0:
            pre_ny_high = round(pre_ny_rates_frame["high"].max(),5)
            pre_ny_low = round(pre_ny_rates_frame["low"].min(),5)
            print(f'\nPre-NY high:.....{pre_ny_high}')
            print(f'Pre-NY low:......{pre_ny_low}')
        else:
            pass
            
        ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 15) & (rates_frame['time'].dt.hour < 19)]
        if len(ny_rates_frame) != 0:    
            ny_high = round(ny_rates_frame["high"].max(),5)
            ny_low = round(ny_rates_frame["low"].min(),5)
            print(f'\nNY high:.........{ny_high}')
            print(f'NY low:..........{ny_low}')
        else:
            pass
        print("__________________________________")
        order_manager.show_orders_and_open_positions()
        
        try:
            #Determine the max and min value of the highs and lows for the last 35 mins
            Last_110 = rates_frame.tail(110)
            MaxHighPrice = Last_110["high"].max()
            MinLowPrice = Last_110["low"].min()
    
            #Determine the if the last 80 mins is in a downtrend or uptrend
            HighestPoint = rates_frame[rates_frame["high"] == MaxHighPrice].iloc[-1]
            LowestPoint = rates_frame[rates_frame["low"] == MinLowPrice].iloc[-1]
        except IndexError as err1:
            print("It is weekend you dumbass! The forex market is closed!")
            
        #Based on the trend (and the conditions inside) decide whether to buy, sell, or wait.
        try:
            print(f'The date and time today is: {datetime.now().strftime("%Y-%m-%d %I:%M %p")}')
            if (HighestPoint["time"] > LowestPoint["time"]) & BEARISH_BIAS:
                print("Looking to sell...\n")
                MaxHighPoint = Last_110[Last_110["high"] == MaxHighPrice].iloc[[0]]
                MaxHighTime = MaxHighPoint['time'].iloc[0]
                rates_frame_p3 = rates_frame[rates_frame["time"]<= MaxHighTime].tail(110)
                p3_rates_frame = rates_frame[rates_frame["time"]>= MaxHighTime]
                Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4 = check_sell_entry_drill()
                print(f'Sell: {Sell}')
                
                if Sell == False:
                    if (len(p1) == 0) and (len(p2) == 0) and (len(p3) == 0):
                        print("Waiting for a proper higher high and higher low to form....\n")
                    elif (len(p1) != 0) and (len(p2) != 0) and (len(p3) != 0):
                        print(f'-----------------------------------\np1: {p1}\n\np2: {p2}\n\np3: {p3} \n-----------------------------------')
                        print("Waiting for a proper BOS...\n")
                    else:
                        pass
                elif Sell == True:
                    p4_is_valid, p4_rates_frame = sell_price_action_detector.verify_p4(p2_bos,p3,p4,p3_rates_frame)
                    orders = mt5.orders_get(symbol=symbol)
                    if orders is None:
                        sell_orders = []
                    else:
                        # Filter only sell orders
                        sell_orders = [order for order in orders if order.type in [3, 5]]
                        
                    current_time = datetime.now() + timedelta(hours=2)
                    one_hour_ago = current_time - timedelta(hours=3)
                    
                    # Convert times to timestamps (seconds since epoch)
                    current_timestamp = int(current_time.timestamp())
                    one_hour_ago_timestamp = int(one_hour_ago.timestamp())
                    
                    positions = mt5.positions_get(symbol=symbol)
                    if positions is None:
                        sell_positions = []
                    else:
                        # Filter only buy orders
                        sell_positions = [
                            position for position in positions
                            if position.type == 1  # Buy position (0 = BUY, 1 = SELL)
                            and one_hour_ago_timestamp <= position.time <= current_timestamp]
                        
                    print(f'-----------------------------------\np1: {p1}\n\np2: {p2}\n\np3: {p3}\n\np4: {p4}\n-----------------------------------')
                    
                    if p4_is_valid and (len(sell_orders) == 0) and (len(sell_positions) == 0):
                        print("P4 is valid...")
                        tick = mt5.symbol_info_tick(symbol)
                        bid_price = tick.bid
                        if bid_price > Entry_price:
                            order_manager.place_sell_stop_order(symbol,Entry_price, SL_price, TP1_price, sell_tp)
                            N = N+1
                        elif bid_price < Entry_price:
                            order_manager.place_sell_limit_order(symbol,Entry_price, SL_price, TP1_price, sell_tp)
                            N = N+1
                        else:
                            print("Wait for more confirmation!")
                                  
                    elif (p4_is_valid == False) and (len(sell_orders) != 0):
                        print("The point p4 got invalidated...")
                        print(f"Found {len(sell_orders)} pending orders.")
                        print("Deleting the existing sell order....")
                        order_manager.delete_order(symbol)
                        N = N-1
                    
                    elif sell_positions != 0: #if there is an open position, have a trailing stop to the top of P5 to be safer.
                        positions = mt5.positions_get(symbol=symbol)
                        if positions is None:
                            positions = []
                        else:
                            # Filter only sell positions
                            sell_positions = [positions for positions in positions if positions.type in [1]]

                        for position in sell_positions:
                            entry_price = position.price_open
                            stop_loss = position.sl
                            tick = mt5.symbol_info_tick(symbol)
                            bid_price = tick.bid
                            new_stop_loss = round(entry_price - 0.00007,5)
                            SL_size = abs(round(stop_loss - entry_price,5))
                            current_RR = (entry_price - bid_price) / SL_size

                            if abs(entry_price - stop_loss) <= 0.00015:
                                if current_RR >=5:
                                    trail_stop_loss(symbol, new_stop_loss)
                                    print(f"Stop loss of {symbol} sell position moved to breakeven!")
                else:
                    print("Dayummm why do you have two p4s??? ")

                orders = mt5.orders_get(symbol=symbol)
                sell_orders = [order for order in orders if order.type in [3, 5]]
                if (len(sell_orders) != 0):
                    print("Waiting for entry...")    
                
                download_and_display_updated_values()
                time.sleep(60)
                continue
            elif (HighestPoint["time"] < LowestPoint["time"]) & BULLISH_BIAS:
                print("Looking to buy...\n")
                MinLowPoint = Last_110[Last_110["low"] == MinLowPrice].iloc[[0]]
                MinLowTime = MinLowPoint['time'].iloc[0]
                rates_frame_p3 = rates_frame[rates_frame["time"]<= MinLowTime].tail(110)
                p3_rates_frame = rates_frame[rates_frame["time"]>= MinLowTime]                
                Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4 = check_buy_entry_drill()
                print(f'Buy: {Buy}')

                if Buy == False:
                    if (len(p1) == 0) and (len(p2) == 0) and (len(p3) == 0):
                        print("Waiting for a proper lower low and lower high to form....\n")
                    elif (len(p1) != 0) and (len(p2) != 0) and (len(p3) != 0):
                        print(f'-----------------------------------\np1: {p1}\n\np2: {p2}\n\np3: {p3} \n-----------------------------------')
                        print("Waiting for a proper BOS...\n")
                    else:
                        pass
                elif Buy == True:
                    p4_is_valid, p4_rates_frame = buy_price_action_detector.verify_p4(p2_bos,p3,p4,p3_rates_frame)
                    orders = mt5.orders_get(symbol=symbol)
                    if orders is None:
                        buy_orders = []
                    else:
                        # Filter only buy orders
                        buy_orders = [order for order in orders if order.type in [2, 4]]

                    current_time = datetime.now() + timedelta(hours=2)
                    one_hour_ago = current_time - timedelta(hours=3)
                    
                    # Convert times to timestamps (seconds since epoch)
                    current_timestamp = int(current_time.timestamp())
                    one_hour_ago_timestamp = int(one_hour_ago.timestamp())
                    
                    positions = mt5.positions_get(symbol=symbol)
                    if positions is None:
                        buy_positions = []
                    else:
                        # Filter only buy orders
                        buy_positions = [
                            position for position in positions
                            if position.type == 0  # Buy position (0 = BUY, 1 = SELL)
                            and one_hour_ago_timestamp <= position.time <= current_timestamp]

                    print(f'-----------------------------------\np1: {p1}\n\np2: {p2}\n\np3: {p3}\n\np4: {p4}\n-----------------------------------')
					
                    if p4_is_valid and (len(buy_orders) == 0) and (len(buy_positions) == 0):
                        print("P4 is valid...")
                        tick = mt5.symbol_info_tick(symbol)
                        ask_price = tick.ask
                        if ask_price > Entry_price:
                            order_manager.place_buy_limit_order(symbol,Entry_price, SL_price, TP1_price, buy_tp)
                            N = N+1
                        elif ask_price < Entry_price:
                            order_manager.place_buy_stop_order(symbol,Entry_price, SL_price, TP1_price, buy_tp)
                            N = N+1
                        else:
                            print("Wait for more confirmation!")
                                  
                    elif (p4_is_valid == False) and (len(buy_orders) != 0):
                        print("The point p4 got invalidated...")
                        print(f"Found {len(buy_orders)} pending orders.")
                        print("Deleting the existing buy order....")
                        order_manager.delete_order(symbol)
                        N = N-1
                    
                    elif len(buy_positions) != 0: #if there is an open position, have a trailing stop to the top of P5 to be safer.
                        positions = mt5.positions_get(symbol=symbol)
                        if positions is None:
                            positions = []
                        else:
                            # Filter only sell positions
                            buy_positions = [positions for positions in positions if positions.type in [0]]

                        for position in buy_positions:
                            entry_price = position.price_open
                            stop_loss = position.sl
                            tick = mt5.symbol_info_tick(symbol)
                            ask_price = tick.ask
                            new_stop_loss = round(entry_price + 0.00007,5)
                            SL_size = abs(round(entry_price - stop_loss,5))
                            current_RR = (ask_price - entry_price) / SL_size

                            if abs(entry_price - stop_loss) <= 0.00015:
                                if current_RR >=5:
                                    trail_stop_loss(symbol, new_stop_loss)
                                    print(f"Stop loss of {symbol} buy position moved to breakeven!")
                else:
                    print("Dayummm why do you have two p4s??? ")

                orders = mt5.orders_get(symbol=symbol)
                buy_orders = [order for order in orders if order.type  in [2, 4]]
                if (len(buy_orders) != 0):
                    print("Waiting for entry...")
                    
                download_and_display_updated_values()
                time.sleep(60)
                continue
            else:
                if BULLISH_BIAS and BEARISH_BIAS:
                    print("Waiting for an opportunity for both buy and sell...")
                elif BULLISH_BIAS:
                    print("Waiting for an opportunity to buy...")
                    orders = mt5.orders_get(symbol=symbol)
                    if orders is None:
                        buy_orders = []
                    else:
                        # Filter only buy orders
                        buy_orders = [order for order in orders if order.type in [2, 4]]
                    if (len(buy_orders) != 0):
                        print("The point p4 got invalidated...")
                        print(f"Found {len(buy_orders)} pending orders.")
                        print('This is a no trade scenario....')
                        order_manager.delete_order(symbol)
                        N = N - 1
                elif BEARISH_BIAS:
                    print("Waiting for an opportunity to sell...")
                    orders = mt5.orders_get(symbol=symbol)
                    if orders is None:
                        sell_orders = []
                    else:
                        # Filter only sell orders
                        sell_orders = [order for order in orders if order.type in [3, 5]]
                    if (len(sell_orders) != 0):
                        print("The point p4 got invalidated...")
                        print(f"Found {len(sell_orders)} pending orders.")
                        print('This is a no trade scenario....')
                        order_manager.delete_order(symbol)
                        N = N - 1
                download_and_display_updated_values()
                time.sleep(60)
                continue
        except NameError as err1:
            print(f'Name error : {err1} ')
            break
        except ValueError as err1:
            print(f'Value error : {err1} ')
            break
        

__________________________________
Pre-London high: 1.2311
Pre-London low:  1.2295
__________________________________
************************************
__________________________________
No pending orders.
__________________________________
************************************
__________________________________
No open positions.
__________________________________
************************************
The date and time today is: 2025-01-23 03:15 PM
Waiting for an opportunity to sell...
Number of trades executed for today: 0
  
GBPUSD OHLC prices from the last 30 minutes :
____________________________________________________________________________
                    time     open     high      low    close
1491 2025-01-23 08:45:00  1.23142  1.23142  1.23128  1.23133
1492 2025-01-23 08:46:00  1.23132  1.23150  1.23130  1.23142
1493 2025-01-23 08:47:00  1.23140  1.23143  1.23133  1.23139
1494 2025-01-23 08:48:00  1.23136  1.23140  1.23130  1.23134
1495 2025-01-23 08:49:00  1.23136  1.